# 분류 분석

## 이진 고객 이탈

고객의 웹사이트 광고를 제공하는 마케팅 대행사에서 당사의 고객이탈율이 꽤 높다는 사실을 알게됐습니다.<br>
그 회사에서는 즉시 고객 관리자들을 할당했지만, 어떤 고객이 이탈 할 것인지 예측하는 기계학습 모델을 만들어서 가장 이탈확률이 높은 고객에게 우선적으로 고객 관리자를 배치할 수 있기를 원합니다.<br>
고객이 잠재적 이탈 고객인지 여부를 분류하는 분류모델을 만드세요.

데이터는 customer_churn.csv로 저장돼있습니다. 아래는 각 컬럼의 정의입니다.
- Name : 회사의 최근 담당자의 이름
- Age : 고객의 나이
- Total_Purchase : 구매한 총 광고
- Account_Manager : 바이너리 0 = 고객 관리자 없음, 1 = 고객 관리자 할당됨
- Years : 거래 유지 연도수
- Num_sites : 서비스를 이용 중인 웹사이트 수
- Onboard_date : 마지막 연락처가 등록된 날짜
- Location : 고객 사무실 주소
- Company : 고객 회사의 이름

모델 생성 후에는 고객이 제공하는 new_customers.csv 파일에 저장된 새로운 데이터에 대한 예측값을 제시하세요. <br>
고객은 이 데이터를 통해 관리가 필요한 고객들을 알고 싶어 합니다.

- cross_validate 적용할 것
    - auc를 계산할 것 (공식 홈페이지에 있으니 찾아봐...)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import RocCurveDisplay, auc
from sklearn.model_selection import StratifiedKFold, cross_val_score
import matplotlib.pyplot as plt
from datetime import date, datetime

### 데이터 불러오기

In [2]:
df = pd.read_csv("./data/customer_churn.csv")

### 데이터 확인

In [3]:
df.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
0,Cameron Williams,42.0,11066.80,0,7.22,8.0,2013-08-30 07:00:40,"10265 Elizabeth Mission Barkerburgh, AK 89518",Harvey LLC,1
1,Kevin Mueller,41.0,11916.22,0,6.50,11.0,2013-08-13 00:38:46,"6157 Frank Gardens Suite 019 Carloshaven, RI 1...",Wilson PLC,1
2,Eric Lozano,38.0,12884.75,0,6.67,12.0,2016-06-29 06:20:07,"1331 Keith Court Alyssahaven, DE 90114","Miller, Johnson and Wallace",1
3,Phillip White,42.0,8010.76,0,6.71,10.0,2014-04-22 12:43:12,"13120 Daniel Mount Angelabury, WY 30645-4695",Smith Inc,1
4,Cynthia Norton,37.0,9191.58,0,5.56,9.0,2016-01-19 15:31:15,"765 Tricia Row Karenshire, MH 71730",Love-Jones,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Names            900 non-null    object 
 1   Age              900 non-null    float64
 2   Total_Purchase   900 non-null    float64
 3   Account_Manager  900 non-null    int64  
 4   Years            900 non-null    float64
 5   Num_Sites        900 non-null    float64
 6   Onboard_date     900 non-null    object 
 7   Location         900 non-null    object 
 8   Company          900 non-null    object 
 9   Churn            900 non-null    int64  
dtypes: float64(4), int64(2), object(4)
memory usage: 70.4+ KB


In [5]:
df.describe()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
count,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000
mean,41.816667,10062.824033,0.481111,5.273156,8.587778,0.166667
std,6.127560,2408.644532,0.499921,1.274449,1.764836,0.372885
min,22.000000,100.000000,0.000000,1.000000,3.000000,0.000000
25%,38.000000,8497.122500,0.000000,4.450000,7.000000,0.000000
50%,42.000000,10045.870000,0.000000,5.215000,8.000000,0.000000
75%,46.000000,11760.105000,1.000000,6.110000,10.000000,0.000000
max,65.000000,18026.010000,1.000000,9.150000,14.000000,1.000000


In [6]:
df["Names"].value_counts()

Names
Jennifer Wood        2
Cameron Williams     1
Stacy Marshall       1
Christopher Clark    1
Rose White           1
                    ..
Ashley Smith         1
Sandra Mcintyre      1
Robert Joseph        1
George Chavez        1
Joseph Williams      1
Name: count, Length: 899, dtype: int64

In [7]:
df["Location"].value_counts()

Location
10265 Elizabeth Mission Barkerburgh, AK 89518            1
4331 Quinn Fords Suite 546 Traceyfort, CO 47194-7606     1
69666 Christian Forges Lake Noah, IL 14397               1
715 Ellis Squares New Kathleenland, NY 05913             1
3004 Nash Freeway North Craigville, CT 24488             1
                                                        ..
3785 Duran Lodge Port Nicholashaven, MH 08892-1815       1
663 White Tunnel Suite 995 Port Joseph, PR 93959-2444    1
10676 Ferguson Ranch Suite 380 Lake Parker, OH 23498     1
95222 Lisa Turnpike South Laurenburgh, NC 33273-8443     1
27252 Olivia Burgs Rivasmouth, MN 80121-6348             1
Name: count, Length: 900, dtype: int64

In [8]:
df["Company"].value_counts()

Company
Anderson Group                     4
Williams PLC                       3
Wilson PLC                         3
Smith Group                        2
Williams LLC                       2
                                  ..
Cowan Inc                          1
Cortez Inc                         1
Rodriguez, Moore and Cunningham    1
Bennett Ltd                        1
Robles-Abbott                      1
Name: count, Length: 873, dtype: int64

### 머신러닝용 데이터 전처리

우리는 수치형 데이터만 이용할 예정입니다. Account_Manager는 다루기 어렵지 않기 때문에 모델학습에 포함할 수도 있겠지만 무작위로 할당된 것이기 때문에 큰 의미가 없을 수 있습니다.

In [9]:
ndf = df.drop(["Names", "Location", "Company", "Account_Manager"], axis = 1)

In [10]:
ndf["Onboard_date"] = ndf["Onboard_date"].str.slice(stop = 10).map(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

In [11]:
today = datetime.today()

In [12]:
ndf["history"] = ndf["Onboard_date"].map(lambda x: today - x).astype("int64")
ndf = ndf.drop("Onboard_date", axis = 1)

In [13]:
ndf.head()

,Age,Total_Purchase,Years,Num_Sites,Churn,history
0,42.0,11066.80,7.22,8.0,1,334152538711617000
1,41.0,11916.22,6.50,11.0,1,335621338711617000
2,38.0,12884.75,6.67,12.0,1,244814938711617000
3,42.0,8010.76,6.71,10.0,1,313848538711617000
4,37.0,9191.58,5.56,9.0,1,258811738711617000


In [14]:
ndf["Purchase/Years"] = round(ndf["Total_Purchase"] / ndf["Years"], 2)

In [15]:
ndf

,Age,Total_Purchase,Years,Num_Sites,Churn,history,Purchase/Years
0,42.0,11066.80,7.22,8.0,1,334152538711617000,1532.80
1,41.0,11916.22,6.50,11.0,1,335621338711617000,1833.26
2,38.0,12884.75,6.67,12.0,1,244814938711617000,1931.75
3,42.0,8010.76,6.71,10.0,1,313848538711617000,1193.85
4,37.0,9191.58,5.56,9.0,1,258811738711617000,1653.16
...,...,...,...,...,...,...,...
895,42.0,12800.82,3.62,8.0,0,515506138711617000,3536.14
896,52.0,9893.92,6.91,7.0,0,481550938711617000,1431.83
897,45.0,12056.18,5.46,4.0,0,308750938711617000,2208.09
898,51.0,6517.93,5.47,10.0,0,373637338711617000,1191.58


In [16]:
x = ndf.drop("Churn", axis = 1)
y = ndf["Churn"]

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 7)

### GradientBoosting 시도

In [172]:
gb = GradientBoostingClassifier(random_state = 7, n_estimators = 250, learning_rate = 0.01)
scores = cross_validate(gb, x_train, y_train, return_train_score = True, n_jobs = -1)

In [173]:
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9190972222222221 0.8763888888888889


In [107]:
cross_val_score(gb, x_train, y_train, scoring="roc_auc").mean()

0.874826388888889

In [108]:
gb.fit(x_train, y_train)

GradientBoostingClassifier(learning_rate=0.2, n_estimators=50, random_state=7)

In [109]:
gb.feature_importances_

array([0.06385398, 0.07599959, 0.14867549, 0.5176731 , 0.11099839,
       0.08279945])

In [174]:
cross_val_score(gb, x, y, scoring="roc_auc").mean()

0.8825333333333333

### ExtraTrees 시도

In [24]:
et = ExtraTreesClassifier(n_jobs = -1, random_state = 7)
scores = cross_validate(et, x_train, y_train, return_train_score = True, n_jobs = -1)

In [25]:
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

1.0 0.8694444444444445


In [26]:
cross_val_score(et, x_train, y_train, scoring="roc_auc").mean()

0.8972569444444446

In [27]:
et.fit(x_train, y_train)

ExtraTreesClassifier(n_jobs=-1, random_state=7)

In [28]:
et.feature_importances_

array([0.11713329, 0.11724694, 0.13826743, 0.37844928, 0.12997775,
       0.11892532])

In [29]:
cross_val_score(et, x, y, scoring="roc_auc").mean()

0.8858444444444444

### RandomForest 시도

In [30]:
rf = RandomForestClassifier(n_jobs = -1, random_state = 7)
scores = cross_validate(rf, x_train, y_train, return_train_score = True, n_jobs = -1)

In [31]:
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

1.0 0.8722222222222221


In [32]:
cross_val_score(rf, x_train, y_train, scoring="roc_auc").mean()

0.881076388888889

In [33]:
rf.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=7)

In [34]:
rf.feature_importances_

array([0.09485793, 0.11739282, 0.14809288, 0.36957054, 0.1412685 ,
       0.12881733])

In [35]:
cross_val_score(rf, x, y, scoring="roc_auc").mean()

0.8833111111111112

### LogisticRegression 시도

In [36]:
ss = StandardScaler()

In [37]:
scaled_train = ss.fit_transform(x_train)
scaled_test = ss.transform(x_test)

In [38]:
logi = LogisticRegression()

In [39]:
logi.fit(scaled_train, y_train)

LogisticRegression()

In [40]:
logi.score(scaled_train, y_train)

0.8944444444444445

In [41]:
logi.score(scaled_test, y_test)

0.9166666666666666

In [42]:
scores = cross_validate(logi, scaled_train, y_train, return_train_score = True, n_jobs = -1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.8944444444444445 0.888888888888889


In [43]:
cross_val_score(logi, scaled_train, y_train, scoring="roc_auc").mean()

0.899375

In [44]:
scores

{'fit_time': array([0.01049638, 0.01049256, 0.01049256, 0.00949049, 0.00949049]),
 'score_time': array([0.00099707, 0.00099945, 0.00099945, 0.00099945, 0.00099945]),
 'test_score': array([0.875     , 0.875     , 0.92361111, 0.90972222, 0.86111111]),
 'train_score': array([0.90104167, 0.90104167, 0.88368056, 0.88888889, 0.89756944])}

### KNN 시도

In [45]:
knn = KNeighborsClassifier(n_neighbors = 5)

In [46]:
knn.fit(scaled_train, y_train)

KNeighborsClassifier()

In [47]:
knn.score(scaled_train, y_train)

0.9236111111111112

In [48]:
knn.score(scaled_test, y_test)

0.8722222222222222

In [49]:
scores = cross_validate(knn, scaled_train, y_train, return_train_score = True, n_jobs = -1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9135416666666666 0.8722222222222221


In [50]:
cross_val_score(knn, scaled_train, y_train, scoring="roc_auc", cv = 5).mean()

0.8517013888888888

# new_customers에 LogisticRegression 적용

In [51]:
target_df = pd.read_csv("./data/new_customers.csv", index_col = False)

In [52]:
target_df.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company
0,Andrew Mccall,37.0,9935.53,1,7.71,8.0,2011-08-29 18:37:54,"38612 Johnny Stravenue Nataliebury, WI 15717-8316",King Ltd
1,Michele Wright,23.0,7526.94,1,9.28,15.0,2013-07-22 18:19:54,"21083 Nicole Junction Suite 332, Youngport, ME...",Cannon-Benson
2,Jeremy Chang,65.0,100.00,1,1.00,15.0,2006-12-11 07:48:13,"085 Austin Views Lake Julialand, WY 63726-4298",Barron-Robertson
3,Megan Ferguson,32.0,6487.50,0,9.40,14.0,2016-10-28 05:32:13,"922 Wright Branch North Cynthialand, NC 64721",Sexton-Golden
4,Taylor Young,32.0,13147.71,1,10.00,8.0,2012-03-20 00:36:46,Unit 0789 Box 0734 DPO AP 39702,Wood LLC


In [53]:
nt_df = target_df.drop(["Names", "Account_Manager", "Location", "Company", "Onboard_date"], axis = 1)

In [54]:
# nt_df["Onboard_date"] = nt_df["Onboard_date"].str.slice(stop = 10).map(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

In [55]:
# nt_df["history"] = nt_df["Onboard_date"].map(lambda x: today - x).astype("int64")
# nt_df = nt_df.drop("Onboard_date", axis = 1)

In [56]:
nt_df["Purchase/Years"] = round(nt_df["Total_Purchase"] / nt_df["Years"], 2)

In [ ]:
nt_df = nt_df.drop("Total_Purchase", axis = 1)

In [152]:
nt_df

,Age,Total_Purchase,Years,Num_Sites,history,Purchase/Years
0,37.0,9935.53,7.71,8.0,397397338711617000,1288.65
1,23.0,7526.94,9.28,15.0,337522138711617000,811.09
2,65.0,100.00,1.00,15.0,546178138711617000,100.00
3,32.0,6487.50,9.40,14.0,234360538711617000,690.16
4,32.0,13147.71,10.00,8.0,379771738711617000,1314.77
5,22.0,8445.26,3.46,14.0,415195738711617000,2440.83


In [58]:
ss = StandardScaler()
ss_x = ss.fit_transform(x)
ss_target = ss.transform(nt_df)

In [59]:
logi.fit(ss_x, y, )

LogisticRegression()

In [60]:
logi.predict(ss_target)

array([0, 1, 1, 1, 0, 1], dtype=int64)

In [61]:
scores = cross_validate(logi, ss_x, y, return_train_score = True, n_jobs = -1)
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.8988888888888888 0.8966666666666667


In [62]:
cross_val_score(logi, ss_x, y, scoring="roc_auc").mean()

0.8956

In [63]:
logi.predict(ss_target)

array([0, 1, 1, 1, 0, 1], dtype=int64)

In [64]:
cross_val_score(knn, ss_x, y, scoring="roc_auc").mean()

0.839488888888889